In [13]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리
import random

## 노드의 블록 정보 확인

In [14]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1671776350.615425,
   'transactions': []}],
 'length': 1}

## transaction 입력하기

In [15]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {"contract_address":"myaddress"}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

## 노드의 블록정보 확인 - 2

In [16]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1671776350.615425,
   'transactions': []}],
 'length': 1}

##  채굴하기 - 1

In [17]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


## 노드의 블록정보 확인 - 3

In [18]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1671776350.615425,
   'transactions': []},
  {'index': 2,
   'nonce': 331633,
   'previous_hash': '25a5c3d5d8db13f7be45610326e64bcff88923f17d917abb7a7dfc962fd9dd7f',
   'timestamp': 1671776421.2989593,
   'transactions': [{'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'smart_contract': {'contract_address': 'myaddress'},
     'timestamp': 1671776417.166413},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'smart_contract': {'contract_address': 'mining_profit'},
     'timestamp': 1671776421.2989593}]}],
 'length': 2}

## transaction 입력하기 -2  / 채굴 / 블록정보 확인

In [25]:
## transaction 입력하기 -2
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to2",
    "amount": 30,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

## transaction3 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to3",
    "amount": 300,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

##  채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드의 블록정보 확인 - 4
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

<Response [200]>


In [26]:
res.text

'{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1670135208.6944222,"transactions":[]},{"index":2,"nonce":-844297,"previous_hash":"7c62c824701f0dbccee9b6c621917e89a69fb8e24087bfaf1054a3a465b5db84","timestamp":1670135305.4202778,"transactions":[{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1670135301.2886367},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1670135305.4202778}]},{"index":3,"nonce":204168,"previous_hash":"9c6be753a014410e60b8e1a41b988fe564382b9e369688ee714f17a3f54d74e4","timestamp":1670135313.691937,"transactions":[{"amount":30,"recipient":"test_to2","sender":"test_from","timestamp":1670135309.5391397},{"amount":300,"recipient":"test_to3","sender":"test_from","timestamp":1670135311.602241},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1670135313.691937}]}],"length":3}\n'

## Pandas 를 활용한 거래내역 조회

In [27]:
status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

,timestamp,sender,recipient,amount
0,1.670135e+09,test_from,test_to,3.0
1,1.670135e+09,master,node_5000,0.1
2,1.670135e+09,test_from,test_to2,30.0
3,1.670135e+09,test_from,test_to3,300.0
4,1.670135e+09,master,node_5000,0.1


## 거래내역 기반 계정별 잔액 조회

In [16]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status

,user,received_amount,sended_amount,balance
0,node_5000,0.3,0.0,0.3
1,test_to,6.0,0.0,6.0
2,test_to2,30.0,0.0,30.0
3,test_to3,300.0,0.0,300.0
4,master,0.0,0.3,-0.3
5,test_from,0.0,336.0,-336.0
